# Skeleton notebook for subject-level and group-level statistics
Use this notebook to write the code for subject- and group-level statistics on EEG data

## Setting up Python
First of all, we need to make sure that we are working in the `env` environment.


1. Run `bash env_to_jupyter.sh` from the `EEG` folder if you have not already done so. This will make sure that the `env` environment is available as a kernel in this notebook.

2. Press `Select Kernel`, then `Jupyter kernel...` and select `env`. If `env` does not show up, press the little refresh symbol!

**Note:** You might have to install the Jupyter extension for VScode to be able to select the kernel.

In [27]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import mne

# Windowed mean

## Subject-level

In [ ]:
# load in the data
data_path = Path("") # insert path to the preprocessed epochs of the FaceWord EEG data
filename = "" # and the filename 

epochs = mne.read_epochs(data_path / filename, verbose=False, preload=True)

# only keep eeg channels
epochs.pick(["eeg"])

Now you are ready to go the `04-windowed_mean_subject_lvl.ipynb` to get inspiration for how to do a subject-level windowed mean analysis! Add code chunks below as needed!

In [ ]:
# code for subject-level windowed mean

## Group-level

The first step is to preprocess the data from all subject. I have set up a loop for doing so and defined a few varibles you may need. **Talk through the code in your groups!**

In [28]:
# We used a different script to run FaceWord experiment on Monday where triggers 101 

# defining the events
event_id_monday_recording = {
    'Word/wPos': 11, # positive word
    'Wait/wPos': 31, # waiting time after positive word 
    'Image/wPos': 21, # positive image (always following pos word) 
    'Word/wNeg': 12, # negative word
    'Wait/wNeg': 32, # waiting time after negative word 
    'Image/wNeg': 22, # negative image (always following neg word) 
    'Word/wNeu': 13, # neutral word
    'Wait/wNeu/iPos': 51, # wait time after neu w (before pos i) 
    'Image/wNeu/iPos': 41, # positive image (after neu word) 
    'Wait/wNeu/iNeg': 52, # wait time after neu w (before neg i) 
    'Image/wNeu/iNeg': 42, # negative image (after neu word) 
    'Correct/wPos': 201, # correct response ('b') to pos w + image 
    'Correct/wNeg': 102, # correct response ('y') to neg w + image 
    'Correct/wNeu/iPos': 111, # cor resp ('b') to neu w + pos image 
    'Correct/wNeu/iNeg': 112, # cor resp ('y') to neu w + neg image 
    'Incorrect/wPos': 202, # incor resp ('y') to pos w + image 
    'Incorrect/wNeg': 101, # incor resp ('b') to neg w + image 
    'Incorrect/wNeu/iPos': 212, # incor resp ('y') to neu w + pos i 
    'Incorrect/Neu/iNeg': 211 # incor resp ('b') to neu w + neg i
}

event_id = {
    'Word/wPos': 11, # positive word
    'Wait/wPos': 31, # waiting time after positive word 
    'Image/wPos': 21, # positive image (always following pos word) 
    'Word/wNeg': 12, # negative word
    'Wait/wNeg': 32, # waiting time after negative word 
    'Image/wNeg': 22, # negative image (always following neg word) 
    'Word/wNeu': 13, # neutral word
    'Wait/wNeu/iPos': 51, # wait time after neu w (before pos i) 
    'Image/wNeu/iPos': 41, # positive image (after neu word) 
    'Wait/wNeu/iNeg': 52, # wait time after neu w (before neg i) 
    'Image/wNeu/iNeg': 42, # negative image (after neu word) 
    'Correct/wPos': 101, # correct response ('b') to pos w + image 
    'Correct/wNeg': 102, # correct response ('y') to neg w + image 
    'Correct/wNeu/iPos': 111, # cor resp ('b') to neu w + pos image 
    'Correct/wNeu/iNeg': 112, # cor resp ('y') to neu w + neg image 
    'Incorrect/wPos': 202, # incor resp ('y') to pos w + image 
    'Incorrect/wNeg': 201, # incor resp ('b') to neg w + image 
    'Incorrect/wNeu/iPos': 212, # incor resp ('y') to neu w + pos i 
    'Incorrect/Neu/iNeg': 211 # incor resp ('b') to neu w + neg i
}

In [29]:
def preprocess_EEG_data(raw, bad_channels:list[str], event_id, tmin = -0.1, tmax = 0.7, baseline = (None, 0), l_freq = 0.1, h_freq = 40.0, reject = {"eeg": 150e-6}):

    raw.info["bads"] = bad_channels


    # setting EOG channels correctly - different names for the channels on the two different acquistion computers
    try:
        raw.set_channel_types({"HEOG": "eog", "VEOG": "eog"})
    
    except(ValueError):
        raw.set_channel_types({"EOG1": "eog", "EOG2": "eog"})

    raw.set_montage("standard_1020")

    

    # interpolate the bad channels 
    # YOU COULD ALSO FIND THE COMMON GOOD CHANNELS WHEN DOING STATS INSTEAD
    # -> BUT WE WILL DO THIS FOR TODAY
    #raw.interpolate_bads(reset_bads=True, mode="accurate", verbose=False)

    # dropping the remaining bad channels after interpolation (Fp1 and Fp2)
    raw.drop_channels(raw.info["bads"])

    raw.filter(l_freq, h_freq, verbose=False)
    
    # setting the reference
    raw.set_eeg_reference("average", projection=False, verbose = False)

    # creating the events
    events, _ = mne.events_from_annotations(raw, verbose=False)

    # remove events from event id that are not in the data (to avoid errors when creating the epochs)
    event_id_tmp = {key: value for key, value in event_id.items() if value in events[:, 2]}


    # creating the epochs
    epochs_tmp = mne.Epochs(
        raw, 
        events, 
        event_id=event_id_tmp,
        tmin=tmin, 
        tmax=tmax, 
        baseline=baseline, 
        preload=True, 
        reject=reject,
        verbose = False
    )

    # downsample the epochs
    epochs_tmp = epochs_tmp.resample(250)

    return epochs_tmp

In [ ]:
# load in the data
data_path = Path("/work/EEG_lab/FaceWord_EEG") # insert path to the preprocessed epochs of the FaceWord EEG data


# loading in the file with the bad channels identified by each group
session_info_path = data_path / "session_info.txt"

    
# load in session information (bad channels, etc.) txt file with dictionary
with open(session_info_path, "r") as f:
    session_info = eval(f.read())

print(session_info)

# creating a figures folder for saving plots from drop log
fig_path = Path("/work/LauraBockPaulsen#1941/CogNeuro2025/EEG_LAB/figures")

if not fig_path.exists():
    fig_path.mkdir(parents=True)

In [ ]:
# Looping over the subjects and preprocessing
all_epochs = []

monday_recording = ["group0-fw2", "group1_fw", "group2_fw", "group3_fw", "group4_fw", "group5_fw"]

for groupname, info in session_info.items():
    print(f"Preprocessing data from {groupname}")
    print(f"Bad channels: {info["bad_channels"]}")

    raw = mne.io.read_raw_brainvision(data_path / f"{groupname}.vhdr", preload=True)

    # check if the raw needs to be cropped
    if info["tmin"] != None or info["tmax"] != None: 
        raw.crop(info["tmin"], info["tmax"])
    
    if groupname in monday_recording:
        event_ids = event_id_monday_recording
    else:
        event_ids = event_ids

    preprocessed_epochs = preprocess_EEG_data(
        raw, 
        bad_channels=info["bad_channels"],
        event_id = event_ids
        )

    drop_log_fig = preprocessed_epochs.plot_drop_log()
    drop_log_fig.savefig(fig_path / f"{groupname}.png")

    all_epochs.append(preprocessed_epochs)

Now you are ready to go the `05-windowed_mean_group_lvl.ipynb` to get inspiration for how to do a group-level windowed mean analysis! Add code chunks below as needed!

In [ ]:
# code for group-level windowed mean

# Cluster-based permutation test
## Subject-level
You have already loaded your data into `epochs`-variable, so you can just use that! Go to the `06-clusterbased_permutation.ipynb` to get inspiration for how to do a subject-level cluster-based permutation test! Add code chunks below as needed!

In [ ]:
# code for subject-level 

## Group-level

Since you already preprocessed the data, you can just use the `all_epochs`-variable defined when preparing for the group-level windowed mean analysis. Go to the `06-clusterbased_permutation.ipynb` to get inspiration for how to do a group-level cluster-based permutation test! Add code chunks below as needed!

In [ ]:
# code for group-level 